# Examples

## Python toolz: `pytlz`

In [1]:
import datetime as dt

from toolz import curried

from onekit import pytlz

### `flatten`

In [2]:
irregular_list = [
    ["one", 2],
    3,
    [(4, "five")],
    [[["six"]]],
    "seven",
    [],
]

list(pytlz.flatten(irregular_list, 8, [9, ("ten",)]))

['one', 2, 3, 4, 'five', 'six', 'seven', 8, 9, 'ten']

### Date computation

#### `daycount`

In [3]:
# month sequence - first date
curried.pipe(
    pytlz.daycount(dt.date(2022, 1, 1)),
    curried.filter(lambda d: d.day == 1),
    curried.map(pytlz.date_to_str),
    curried.take(5),
    list,
)

['2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01']

In [4]:
# month sequence - last date
curried.pipe(
    pytlz.daycount(dt.date(2022, 1, 1)),
    curried.filter(lambda d: d.day == 1),
    curried.map(lambda d: pytlz.last_date_of_month(d.year, d.month)),
    curried.map(pytlz.date_to_str),
    curried.take(5),
    list,
)

['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31']

In [5]:
# Monday sequence
curried.pipe(
    pytlz.daycount(dt.date(2022, 1, 1)),
    curried.filter(lambda d: pytlz.weekday(d) == "Mon"),
    curried.map(pytlz.date_to_str),
    curried.take(5),
    list,
)

['2022-01-03', '2022-01-10', '2022-01-17', '2022-01-24', '2022-01-31']

In [6]:
# pick every 7th day
curried.pipe(
    pytlz.daycount(dt.date(2022, 1, 1)),
    curried.take_nth(7),
    curried.map(pytlz.date_to_str),
    curried.take(5),
    list,
)

['2022-01-01', '2022-01-08', '2022-01-15', '2022-01-22', '2022-01-29']

Task: The digits of 22 February 2022 form [a palindrome and an ambigram](https://en.wikipedia.org/wiki/Twosday) in dd-mm-yyyy format.
List the next five dates with these properties.

In [7]:
def format_date(d: dt.date, with_hyphen: bool) -> str:
    fmt = "%d-%m-%Y" if with_hyphen else "%d%m%Y"
    return d.strftime(fmt)


def is_palindrome_date(d: dt.date) -> bool:
    d_str = format_date(d, False)
    return d_str == d_str[::-1]


def is_ambigram_date(d: dt.date) -> bool:
    d_str = format_date(d, False)
    return set(d_str) <= {"0", "1", "2", "8"}


def show_date(d: dt.date) -> str:
    return f"{format_date(d, True)} ↦ {format_date(d, False)}"


curried.pipe(
    pytlz.daycount(dt.date(2022, 2, 23)),
    curried.filter(is_palindrome_date),
    curried.filter(is_ambigram_date),
    curried.map(show_date),
    curried.take(5),
    list,
)

['08-02-2080 ↦ 08022080',
 '18-02-2081 ↦ 18022081',
 '28-02-2082 ↦ 28022082',
 '10-12-2101 ↦ 10122101',
 '20-12-2102 ↦ 20122102']

#### `daterange`

In [8]:
# month sequence - first date
curried.pipe(
    pytlz.daterange(dt.date(2022, 1, 1), dt.date(2022, 5, 31)),
    curried.filter(lambda d: d.day == 1),
    curried.map(pytlz.date_to_str),
    curried.take(5),
    list,
)

['2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01']

In [9]:
# month sequence - last date
curried.pipe(
    pytlz.daterange(dt.date(2022, 1, 1), dt.date(2022, 5, 31)),
    curried.filter(lambda d: d.day == 1),
    curried.map(lambda d: pytlz.last_date_of_month(d.year, d.month)),
    curried.map(pytlz.date_to_str),
    curried.take(5),
    list,
)

['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31']

## PySpark toolz: `sparktlz`

In [10]:
import os

from pyspark.sql import SparkSession

from onekit import sparktlz

In [11]:
spark = (
    SparkSession.builder.master("local[1]")
    .appName("spark-session-docs")
    .config("spark.sql.shuffle.partitions", 1)
    .config("spark.default.parallelism", os.cpu_count())
    .config("spark.rdd.compress", False)
    .config("spark.shuffle.compress", False)
    .config("spark.dynamicAllocation.enabled", False)
    .config("spark.executor.cores", 1)
    .config("spark.executor.instances", 1)
    .config("spark.ui.enabled", False)
    .config("spark.ui.showConsoleProgress", False)
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

23/11/17 21:56:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [12]:
df = sparktlz.union(
    spark.createDataFrame([dict(x=1, y=2), dict(x=3, y=4)]),
    spark.createDataFrame([dict(x=5, y=6), dict(x=7, y=8)]),
    spark.createDataFrame([dict(x=0, y=1), dict(x=2, y=3)]),
)

In [13]:
df.show()

+---+---+
|  x|  y|
+---+---+
|  1|  2|
|  3|  4|
|  5|  6|
|  7|  8|
|  0|  1|
|  2|  3|
+---+---+



In [14]:
spark.stop()